In [93]:
import math
import torch
from torch import nn
from d2l import torch as d2l

#### tensor.transpose

In [94]:
X = torch.arange(12*2)
X = X.reshape(2, 3, 4)
X

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]])

In [95]:
# 在 X 的 1 和 2 的维度上进行转置
# 写成 X.transpose(2, 1) 也是等价的
# transpose 只是表明再哪两个维度所组成的矩阵上转置
# 并不能表明转之后的顺序
# 转置只能对矩阵进行
X.transpose(1, 2)

tensor([[[ 0,  4,  8],
         [ 1,  5,  9],
         [ 2,  6, 10],
         [ 3,  7, 11]],

        [[12, 16, 20],
         [13, 17, 21],
         [14, 18, 22],
         [15, 19, 23]]])

In [96]:
def masked_softmax(X, valid_lens):
    """Perform softmax operation by masking elements on the last axis.

    Defined in :numref:`sec_attention-scoring-functions`"""
    # `X`: 3D tensor, `valid_lens`: 1D or 2D tensor
    # X (batch_size, num_steps, vector_size)
    if valid_lens is None:
        return nn.functional.softmax(X, dim=-1)
    else:
        shape = X.shape
        if valid_lens.dim() == 1:
            valid_lens = torch.repeat_interleave(valid_lens, shape[1])
        else:
            # 给定的 valid_lens 是一个矩阵
            # 那么 对于第 j 个序列 且 第 i 个 q，只要前 valid_lens[j][i] 个得分
            valid_lens = valid_lens.reshape(-1)
        # On the last axis, replace masked elements with a very large negative
        # value, whose exponentiation outputs 0
        # value=-1e6 是一个绝对值特别大的负数，即 -100 万
        # 经过一层这样的 mask 之后再计算 softmax 就可以让对应位置的概率得 0 了
        X = d2l.sequence_mask(X.reshape(-1, shape[-1]), valid_lens,
                              value=-1e6)

        # 在 sequence_mask 之前是先把三维的数据展平成了一个二维的数据
        # 让 三维 数据中后两个维度组成的矩阵按照第 1 个维度的顺序依次向下堆叠拼接
        # 然后再使用 sequence_mask 对它进行 mask 操作
        # 最后 对 mask 之后的数据进行一个 softmax 运算
        # dim = -1 就是在最后一个维度所组成的向量上进行 softmax 运算
        return nn.functional.softmax(X.reshape(shape), dim=-1)

In [97]:
class DotProductAttention(nn.Module):
    """Scaled dot product attention.

    Defined in :numref:`subsec_additive-attention`"""
    def __init__(self, dropout, **kwargs):
        super(DotProductAttention, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)

    # Shape of `queries`: (`batch_size`, no. of queries, `d`)
    # Shape of `keys`: (`batch_size`, no. of key-value pairs, `d`)
    # Shape of `values`: (`batch_size`, no. of key-value pairs, value
    # dimension)
    # Shape of `valid_lens`: (`batch_size`,) or (`batch_size`, no. of queries)
    def forward(self, queries, keys, values, valid_lens=None):
        d = queries.shape[-1]
        # Set `transpose_b=True` to swap the last two dimensions of `keys`
        # 转之后就可以让 queries 中的每一个 batch 且 batch 中每一个长度为 d 的行向量
        # 与右侧的 每一个 batch 中的每一个长度为 d 的列向量进行运算
        # 矩阵和矩阵的运算，然后堆叠在一起形成一个三位的输出结果
        # tip: 每个矩阵是一个样本 batch_size 个矩阵就代表有 batch_size 个样本
        # 而序列类型处理中关注的是当前序列的前后关系，并不关注当前序列中某个元素和其他序列中的关系
        scores = torch.bmm(queries, keys.transpose(1,2)) / math.sqrt(d)
        print('scores.shape is ', scores.shape)
        print(scores)
        # scores 的每一行是一个 q 向量和所有的 k 向量做完点积后的得分
        # 所以 valid_lens 要指出每一行上的得分要前几个
        # 如果 valid_lens 是一个矩阵并且每一行都是 vec = [1, 2, 3 ... num_steps]
        # 这就代表对于第 i 个 q，它只要前 vec[i] 个得分做 softmax
        self.attention_weights = masked_softmax(scores, valid_lens)

        # values 中某个样本矩阵中 每一行是一个 value 的向量表示
        # 因此矩阵运算左侧的 attention_weight 矩阵每行都是对各个 value 向量的归一化后的分数
        # attention_weight 的每行是一个概率分布，意义对应的乘在右侧的每个行向量上
        # 最终加和在一起称为一个输出向量
        print('attention_weights.shape is ', self.attention_weights.shape)
        print(self.attention_weights)
        return torch.bmm(self.dropout(self.attention_weights), values)

In [98]:
#@save
def transpose_qkv(X, num_heads):
    """为了多注意力头的并行计算而变换形状"""
    # 输入X的形状:(batch_size，查询或者“键－值”对的个数，num_hiddens)
    # 输出X的形状:(batch_size，查询或者“键－值”对的个数，num_heads，
    # num_hiddens/num_heads)
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)

    # 输出X的形状:(batch_size，num_heads，查询或者“键－值”对的个数,
    # num_hiddens/num_heads)
    # num_hiddens/num_heads = 每个 head 的输入长度
    X = X.permute(0, 2, 1, 3)

    # 最终输出的形状:(batch_size*num_heads,查询或者“键－值”对的个数,
    # num_hiddens/num_heads)
    return X.reshape(-1, X.shape[2], X.shape[3])


#@save
def transpose_output(X, num_heads):
    """逆转transpose_qkv函数的操作"""
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)

In [99]:
#@save
class MultiHeadAttention(nn.Module):
    """多头注意力"""
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 num_heads, dropout, bias=False, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.attention = DotProductAttention(dropout)

        # 按道理来说，每个 head 内部都应该有一个 W_q, W_k, W_v
        # 这里为了数据处理方便，把每个 head 内部 科学系的矩阵合并在了一起
        # 从而有了下方的这三个矩阵
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)

        # W_o 用来规范化输出长度
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)

    def forward(self, queries, keys, values, valid_lens):
        # queries，keys，values的形状:
        # (batch_size，查询或者“键－值”对的个数，num_hiddens)
        # valid_lens　的形状:
        # (batch_size，)或(batch_size，查询的个数)
        # 经过变换后，输出的queries，keys，values　的形状:
        # (batch_size*num_heads，查询或者“键－值”对的个数，
        # num_hiddens/num_heads)
        # 反正做 Self-Attention 的 q 和 k 配对的时候是对每个样本序列进行计算
        # 并且在使用 DotProduct 并不会学习任何参数
        # 因此在多个 head 中计算 q 和 k 使用的计算逻辑是一样的，但是输入的样本是不一样的
        # 不妨把多个 head 的计算统计使用同一个计算模块来代替
        # 这样一来就好像是有 num_heads * batch_size 这么多个样本一样 反正样本和样本之间也没有任何交集和干扰
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        values = transpose_qkv(self.W_v(values), self.num_heads)

        if valid_lens is not None:
            # 在轴0，将第一项（标量或者矢量）复制num_heads次，
            # 然后如此复制第二项，然后诸如此类。
            # 因为每个 head 都需要一个 valid_lens
            # 因此就复制 num_heads 次 valid_lens
            valid_lens = torch.repeat_interleave(
                valid_lens, repeats=self.num_heads, dim=0)

        # output的形状:(batch_size*num_heads，查询的个数，
        # num_hiddens/num_heads) num_hiddens/num_heads = 计算得分时 qkv 的向量长度
        output = self.attention(queries, keys, values, valid_lens)

        # output_concat的形状:(batch_size，查询的个数，num_hiddens)
        # 这里的 output_concat 最后的一个维度是 num_hiddens
        # 这就相当于把所有 head 的输出拼接在了一起，最后再丢给 W_o 做一个输出格式的变换
        output_concat = transpose_output(output, self.num_heads)
        return self.W_o(output_concat)

In [100]:
num_hiddens, num_heads = 100, 5
attention = MultiHeadAttention(num_hiddens, num_hiddens, num_hiddens,
                               num_hiddens, num_heads, 0.5)
attention.eval()

MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

In [101]:
batch_size, num_queries = 2, 4


num_kvpairs, valid_lens =  6, torch.tensor([3, 2])
X = torch.ones((batch_size, num_queries, num_hiddens))
Y = torch.ones((batch_size, num_kvpairs, num_hiddens))
attention(X, Y, Y, valid_lens).shape

scores.shape is  torch.Size([10, 4, 6])
tensor([[[ 0.2754,  0.2754,  0.2754,  0.2754,  0.2754,  0.2754],
         [ 0.2754,  0.2754,  0.2754,  0.2754,  0.2754,  0.2754],
         [ 0.2754,  0.2754,  0.2754,  0.2754,  0.2754,  0.2754],
         [ 0.2754,  0.2754,  0.2754,  0.2754,  0.2754,  0.2754]],

        [[-0.7142, -0.7142, -0.7142, -0.7142, -0.7142, -0.7142],
         [-0.7142, -0.7142, -0.7142, -0.7142, -0.7142, -0.7142],
         [-0.7142, -0.7142, -0.7142, -0.7142, -0.7142, -0.7142],
         [-0.7142, -0.7142, -0.7142, -0.7142, -0.7142, -0.7142]],

        [[ 0.0615,  0.0615,  0.0615,  0.0615,  0.0615,  0.0615],
         [ 0.0615,  0.0615,  0.0615,  0.0615,  0.0615,  0.0615],
         [ 0.0615,  0.0615,  0.0615,  0.0615,  0.0615,  0.0615],
         [ 0.0615,  0.0615,  0.0615,  0.0615,  0.0615,  0.0615]],

        [[ 0.1407,  0.1407,  0.1407,  0.1407,  0.1407,  0.1407],
         [ 0.1407,  0.1407,  0.1407,  0.1407,  0.1407,  0.1407],
         [ 0.1407,  0.1407,  0.1407,  0.1407

torch.Size([2, 4, 100])

In [102]:
d2l.sequence_mask(X, valid_lens)[:, :, :5]

tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])

In [103]:
# 维度不变，把每个数值复制 n 次
n = 3
torch.repeat_interleave(valid_lens, n)

tensor([3, 3, 3, 2, 2, 2])

In [104]:
X = torch.arange(1, 6).repeat(2, 1)
X

tensor([[1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5]])

In [105]:
X.repeat(2, 1)

tensor([[1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5]])

In [106]:
torch.repeat_interleave(X, 2, dim=0)

tensor([[1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5]])

#### softmax

通过下方的一系列例子，可以表明 softmax 默认在最后一个维度上进行归一化，其中 dim 参数指明在哪个维度进行归一化，并且归一化后，值为 0 的情况并不会被归一化为 0，softmax 只是让小的更小，大的更大，这里的更小和更大只是让两者在差异上显得更大而已，但是为 0 的值归一化后却不是 0

In [107]:
X = torch.arange(2*3*4, dtype=torch.float32).reshape(2, 3, 4)
X

tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]],

        [[12., 13., 14., 15.],
         [16., 17., 18., 19.],
         [20., 21., 22., 23.]]])

In [108]:
torch.nn.functional.softmax(X, dim=-1)

tensor([[[0.0321, 0.0871, 0.2369, 0.6439],
         [0.0321, 0.0871, 0.2369, 0.6439],
         [0.0321, 0.0871, 0.2369, 0.6439]],

        [[0.0321, 0.0871, 0.2369, 0.6439],
         [0.0321, 0.0871, 0.2369, 0.6439],
         [0.0321, 0.0871, 0.2369, 0.6439]]])

In [109]:
torch.nn.functional.softmax(torch.tensor([0, 2, 2], dtype=torch.float32), dim=-1)

tensor([0.0634, 0.4683, 0.4683])

In [110]:
torch.nn.functional.softmax(torch.tensor([ 0.,  1.,  2.,  3.], dtype=torch.float32), dim=-1)

tensor([0.0321, 0.0871, 0.2369, 0.6439])

In [111]:
torch.nn.functional.softmax(torch.tensor([ 4.,  5.,  6.,  7.], dtype=torch.float32), dim=-1)

tensor([0.0321, 0.0871, 0.2369, 0.6439])

In [112]:
torch.nn.functional.softmax(X, dim=0)

tensor([[[6.1442e-06, 6.1442e-06, 6.1442e-06, 6.1442e-06],
         [6.1442e-06, 6.1442e-06, 6.1442e-06, 6.1442e-06],
         [6.1442e-06, 6.1442e-06, 6.1442e-06, 6.1442e-06]],

        [[9.9999e-01, 9.9999e-01, 9.9999e-01, 9.9999e-01],
         [9.9999e-01, 9.9999e-01, 9.9999e-01, 9.9999e-01],
         [9.9999e-01, 9.9999e-01, 9.9999e-01, 9.9999e-01]]])

In [113]:
torch.nn.functional.softmax(torch.tensor([0, 12], dtype=torch.float32), dim=-1)

tensor([6.1442e-06, 9.9999e-01])

In [114]:
-2.5079e1

-25.079

In [115]:
-1e6

-1000000.0

#### 理解 sequence_mask

In [117]:
X = torch.tensor([
    [1, 2, 3, 4],
    [1, 2, 3, 4],
    [1, 2, 3, 4],
    [1, 2, 3, 4]
])

valid = torch.tensor([2, 3, 2, 1])

d2l.sequence_mask(X, valid)

tensor([[1, 2, 0, 0],
        [1, 2, 3, 0],
        [1, 2, 0, 0],
        [1, 0, 0, 0]])